<a href="https://colab.research.google.com/github/Swagat1342/Enhanced-Stock-Prediction-System-with-Risk-Management/blob/main/finance_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas yfinance scikit-learn tensorflow plotly

In [1]:
pip install alpha_vantage finnhub-python polygon-api-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.2/170.2 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.15.1 requires websockets<16.0.0,>=15.0.1, but you have websockets 14.2 which is incompatible.


In [ ]:
pip install --upgrade yfinance


In [2]:
pip install yfinance ta pandas numpy plotly


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2931963cf5119edf547abcd3399eb8ceed160b278ad71a5f13bfdd782b9ac9f2
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
# IMPORTS
# ==========================================
import warnings
warnings.filterwarnings('ignore')

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [4]:
# ==========================================
# INDIAN STOCKS DATABASE
# ==========================================
INDIAN_STOCKS = {
    # Nifty 50 Stocks
    'NIFTY_50': {
        'RELIANCE.NS': 'Reliance Industries',
        'TCS.NS': 'Tata Consultancy Services',
        'HDFCBANK.NS': 'HDFC Bank',
        'INFY.NS': 'Infosys',
        'ICICIBANK.NS': 'ICICI Bank',
        'HINDUNILVR.NS': 'Hindustan Unilever',
        'ITC.NS': 'ITC Limited',
        'SBIN.NS': 'State Bank of India',
        'BHARTIARTL.NS': 'Bharti Airtel',
        'KOTAKBANK.NS': 'Kotak Mahindra Bank',
        'LT.NS': 'Larsen & Toubro',
        'AXISBANK.NS': 'Axis Bank',
        'BAJFINANCE.NS': 'Bajaj Finance',
        'ASIANPAINT.NS': 'Asian Paints',
        'MARUTI.NS': 'Maruti Suzuki',
        'HCLTECH.NS': 'HCL Technologies',
        'WIPRO.NS': 'Wipro',
        'ULTRACEMCO.NS': 'UltraTech Cement',
        'TITAN.NS': 'Titan Company',
        'NESTLEIND.NS': 'Nestle India',
        'SUNPHARMA.NS': 'Sun Pharmaceutical',
        'TECHM.NS': 'Tech Mahindra',
        'POWERGRID.NS': 'Power Grid Corporation',
        'NTPC.NS': 'NTPC Limited',
        'M&M.NS': 'Mahindra & Mahindra',
        'TATAMOTORS.NS': 'Tata Motors',
        'BAJAJFINSV.NS': 'Bajaj Finserv',
        'ADANIPORTS.NS': 'Adani Ports',
        'COALINDIA.NS': 'Coal India',
        'ONGC.NS': 'Oil and Natural Gas Corporation',
        'TATASTEEL.NS': 'Tata Steel',
        'DIVISLAB.NS': 'Divi\'s Laboratories',
        'DRREDDY.NS': 'Dr. Reddy\'s Laboratories',
        'GRASIM.NS': 'Grasim Industries',
        'HINDALCO.NS': 'Hindalco Industries',
        'INDUSINDBK.NS': 'IndusInd Bank',
        'JSWSTEEL.NS': 'JSW Steel',
        'EICHERMOT.NS': 'Eicher Motors',
        'HEROMOTOCO.NS': 'Hero MotoCorp',
        'CIPLA.NS': 'Cipla',
        'BRITANNIA.NS': 'Britannia Industries',
        'SHREECEM.NS': 'Shree Cement',
        'UPL.NS': 'UPL Limited',
        'APOLLOHOSP.NS': 'Apollo Hospitals',
        'BPCL.NS': 'Bharat Petroleum',
        'ADANIENT.NS': 'Adani Enterprises',
        'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TATACONSUM.NS': 'Tata Consumer Products',
        'SBILIFE.NS': 'SBI Life Insurance',
        'HDFCLIFE.NS': 'HDFC Life Insurance',
    },

    # Banking Sector
    'BANKING': {
        'HDFCBANK.NS': 'HDFC Bank',
        'ICICIBANK.NS': 'ICICI Bank',
        'SBIN.NS': 'State Bank of India',
        'KOTAKBANK.NS': 'Kotak Mahindra Bank',
        'AXISBANK.NS': 'Axis Bank',
        'INDUSINDBK.NS': 'IndusInd Bank',
        'BANDHANBNK.NS': 'Bandhan Bank',
        'FEDERALBNK.NS': 'Federal Bank',
        'IDFCFIRSTB.NS': 'IDFC First Bank',
        'PNB.NS': 'Punjab National Bank',
        'BANKBARODA.NS': 'Bank of Baroda',
        'CANBK.NS': 'Canara Bank',
    },

    # IT Sector
    'IT': {
        'TCS.NS': 'Tata Consultancy Services',
        'INFY.NS': 'Infosys',
        'HCLTECH.NS': 'HCL Technologies',
        'WIPRO.NS': 'Wipro',
        'TECHM.NS': 'Tech Mahindra',
        'LTIM.NS': 'LTIMindtree',
        'PERSISTENT.NS': 'Persistent Systems',
        'COFORGE.NS': 'Coforge',
        'MPHASIS.NS': 'Mphasis',
    },

    # Pharma Sector
    'PHARMA': {
        'SUNPHARMA.NS': 'Sun Pharmaceutical',
        'DIVISLAB.NS': 'Divi\'s Laboratories',
        'DRREDDY.NS': 'Dr. Reddy\'s Laboratories',
        'CIPLA.NS': 'Cipla',
        'AUROPHARMA.NS': 'Aurobindo Pharma',
        'LUPIN.NS': 'Lupin',
        'BIOCON.NS': 'Biocon',
        'TORNTPHARM.NS': 'Torrent Pharmaceuticals',
        'ALKEM.NS': 'Alkem Laboratories',
    },

    # Auto Sector
    'AUTO': {
        'MARUTI.NS': 'Maruti Suzuki',
        'TATAMOTORS.NS': 'Tata Motors',
        'M&M.NS': 'Mahindra & Mahindra',
        'EICHERMOT.NS': 'Eicher Motors',
        'HEROMOTOCO.NS': 'Hero MotoCorp',
        'BAJAJ-AUTO.NS': 'Bajaj Auto',
        'TVSMOTOR.NS': 'TVS Motor',
        'ASHOKLEY.NS': 'Ashok Leyland',
        'MOTHERSON.NS': 'Motherson Sumi',
    },

    # FMCG Sector
    'FMCG': {
        'HINDUNILVR.NS': 'Hindustan Unilever',
        'ITC.NS': 'ITC Limited',
        'NESTLEIND.NS': 'Nestle India',
        'BRITANNIA.NS': 'Britannia Industries',
        'DABUR.NS': 'Dabur India',
        'MARICO.NS': 'Marico',
        'GODREJCP.NS': 'Godrej Consumer Products',
        'COLPAL.NS': 'Colgate-Palmolive',
        'TATACONSUM.NS': 'Tata Consumer Products',
    },

    # Energy & Power
    'ENERGY': {
        'RELIANCE.NS': 'Reliance Industries',
        'ONGC.NS': 'ONGC',
        'BPCL.NS': 'Bharat Petroleum',
        'IOC.NS': 'Indian Oil Corporation',
        'NTPC.NS': 'NTPC',
        'POWERGRID.NS': 'Power Grid Corporation',
        'COALINDIA.NS': 'Coal India',
        'GAIL.NS': 'GAIL India',
        'ADANIGREEN.NS': 'Adani Green Energy',
        'TATAPOWER.NS': 'Tata Power',
    },

    # Metals & Mining
    'METALS': {
        'TATASTEEL.NS': 'Tata Steel',
        'HINDALCO.NS': 'Hindalco Industries',
        'JSWSTEEL.NS': 'JSW Steel',
        'COALINDIA.NS': 'Coal India',
        'VEDL.NS': 'Vedanta',
        'SAIL.NS': 'SAIL',
        'NMDC.NS': 'NMDC',
        'HINDZINC.NS': 'Hindustan Zinc',
    },

    # Infrastructure
    'INFRA': {
        'LT.NS': 'Larsen & Toubro',
        'ADANIPORTS.NS': 'Adani Ports',
        'ULTRACEMCO.NS': 'UltraTech Cement',
        'SHREECEM.NS': 'Shree Cement',
        'GRASIM.NS': 'Grasim Industries',
        'AMBUJACEM.NS': 'Ambuja Cements',
        'ACC.NS': 'ACC Limited',
    },

    # Telecom
    'TELECOM': {
        'BHARTIARTL.NS': 'Bharti Airtel',
        'IDEA.NS': 'Vodafone Idea',
    },

    # Midcap Favorites
    'MIDCAP': {
        'PAGEIND.NS': 'Page Industries',
        'PIDILITIND.NS': 'Pidilite Industries',
        'BERGEPAINT.NS': 'Berger Paints',
        'HAVELLS.NS': 'Havells India',
        'VOLTAS.NS': 'Voltas',
        'GODREJPROP.NS': 'Godrej Properties',
        'DLF.NS': 'DLF Limited',
        'OBEROIRLTY.NS': 'Oberoi Realty',
        'PHOENIXLTD.NS': 'Phoenix Mills',
    }
}


In [5]:
# Flatten all stocks into one dictionary
ALL_INDIAN_STOCKS = {}
for sector_stocks in INDIAN_STOCKS.values():
    ALL_INDIAN_STOCKS.update(sector_stocks)

# ==========================================
# CONFIGURATION
# ==========================================
API_KEYS = {
    'ALPHA_VANTAGE': "YOUR_ALPHA_VANTAGE_API_KEY",
    'IEX_CLOUD': "YOUR_IEX_CLOUD_API_KEY",
    'FINNHUB': "YOUR_FINNHUB_API_KEY",
    'POLYGON': "YOUR_POLYGON_API_KEY"
}


In [6]:
# ==========================================
# UTILITY FUNCTIONS FOR INDIAN STOCKS
# ==========================================
def list_sectors():
    """Display all available sectors"""
    print("\n" + "="*60)
    print("📊 AVAILABLE INDIAN STOCK SECTORS")
    print("="*60)
    for sector, stocks in INDIAN_STOCKS.items():
        print(f"\n{sector}: {len(stocks)} stocks")
    print(f"\nTotal Indian Stocks: {len(ALL_INDIAN_STOCKS)}")

def list_stocks_by_sector(sector=None):
    """List all stocks in a sector or all sectors"""
    if sector:
        sector = sector.upper()
        if sector in INDIAN_STOCKS:
            print(f"\n{'='*60}")
            print(f"📈 {sector} SECTOR STOCKS")
            print(f"{'='*60}")
            for ticker, name in INDIAN_STOCKS[sector].items():
                print(f"{ticker:<20} - {name}")
        else:
            print(f"❌ Sector '{sector}' not found. Use list_sectors() to see available sectors.")
    else:
        print(f"\n{'='*60}")
        print("📊 ALL INDIAN STOCKS")
        print(f"{'='*60}")
        for sector, stocks in INDIAN_STOCKS.items():
            print(f"\n{sector}:")
            for ticker, name in stocks.items():
                print(f"  {ticker:<20} - {name}")

def search_stock(query):
    """Search for stocks by name or ticker"""
    query = query.upper()
    results = []

    for ticker, name in ALL_INDIAN_STOCKS.items():
        if query in ticker.upper() or query in name.upper():
            results.append((ticker, name))

    if results:
        print(f"\n{'='*60}")
        print(f"🔍 SEARCH RESULTS FOR: '{query}'")
        print(f"{'='*60}")
        for ticker, name in results:
            print(f"{ticker:<20} - {name}")
    else:
        print(f"\n❌ No stocks found matching '{query}'")

    return results

def get_stock_info(ticker):
    """Get stock information"""
    ticker = ticker.upper()
    if ticker in ALL_INDIAN_STOCKS:
        return {
            'ticker': ticker,
            'name': ALL_INDIAN_STOCKS[ticker],
            'sector': next((sector for sector, stocks in INDIAN_STOCKS.items() if ticker in stocks), 'Unknown')
        }
    else:
        return None


In [7]:
# ==========================================
# STOCK DATA FETCHER CLASS
# ==========================================
class StockDataFetcher:
    """Fetch stock data from multiple sources with automatic fallback"""

    def __init__(self, api_keys=None):
        self.api_keys = api_keys or API_KEYS
        self.session = self._make_session()
        self.last_source = None

    def _make_session(self):
        """Create HTTP session with retry logic"""
        session = requests.Session()
        retries = Retry(
            total=3,
            backoff_factor=1,
            status_forcelist=[429, 500, 502, 503, 504]
        )
        session.mount('https://', HTTPAdapter(max_retries=retries))
        return session

    def fetch_alpha_vantage(self, symbol, interval='5min', outputsize='compact'):
        """Fetch from Alpha Vantage API"""
        print(f"\n🔹 Trying Alpha Vantage for {symbol}...")
        try:
            url = (
                f"https://www.alphavantage.co/query?"
                f"function=TIME_SERIES_INTRADAY&symbol={symbol}"
                f"&interval={interval}&apikey={self.api_keys['ALPHA_VANTAGE']}"
                f"&outputsize={outputsize}"
            )
            r = self.session.get(url, timeout=10)
            data = r.json()
            ts_key = f"Time Series ({interval})"

            if ts_key not in data:
                print(f"❌ Alpha Vantage Error: {data.get('Note', data.get('Error Message', 'Unknown'))}")
                return None

            df = pd.DataFrame.from_dict(data[ts_key], orient="index").astype(float)
            df.index = pd.to_datetime(df.index)
            df.rename(columns={
                "1. open": "open", "2. high": "high",
                "3. low": "low", "4. close": "close",
                "5. volume": "volume"
            }, inplace=True)

            print(f"✅ Alpha Vantage Success – {len(df)} records")
            self.last_source = "Alpha Vantage"
            return df.sort_index()
        except Exception as e:
            print(f"❌ Alpha Vantage failed: {e}")
            return None

    def fetch_polygon(self, symbol, start_date=None, end_date=None, timespan='minute', multiplier=5):
        """Fetch from Polygon.io API"""
        print(f"\n🔹 Trying Polygon.io for {symbol}...")
        try:
            if not start_date:
                start_date = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
            if not end_date:
                end_date = datetime.now().strftime('%Y-%m-%d')

            url = (
                f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/"
                f"{multiplier}/{timespan}/{start_date}/{end_date}?"
                f"apiKey={self.api_keys['POLYGON']}"
            )
            r = self.session.get(url, timeout=10)
            data = r.json()

            if "results" not in data:
                print(f"❌ Polygon.io Error: {data.get('error', 'No results')}")
                return None

            df = pd.DataFrame(data["results"])
            df["t"] = pd.to_datetime(df["t"], unit="ms")
            df.rename(columns={
                "t": "timestamp", "o": "open", "h": "high",
                "l": "low", "c": "close", "v": "volume"
            }, inplace=True)
            df.set_index("timestamp", inplace=True)

            print(f"✅ Polygon.io Success – {len(df)} records")
            self.last_source = "Polygon.io"
            return df
        except Exception as e:
            print(f"❌ Polygon.io failed: {e}")
            return None

    def fetch_finnhub(self, symbol):
        """Fetch from Finnhub API"""
        print(f"\n🔹 Trying Finnhub for {symbol}...")
        try:
            url = f"https://finnhub.io/api/v1/quote?symbol={symbol}&token={self.api_keys['FINNHUB']}"
            r = self.session.get(url, timeout=10)
            data = r.json()

            if "c" not in data or data.get("c") == 0:
                print(f"❌ Finnhub Error: No data available")
                return None

            df = pd.DataFrame([{
                "open": data.get("o"),
                "high": data.get("h"),
                "low": data.get("l"),
                "close": data.get("c"),
                "volume": None
            }], index=[datetime.now()])

            print(f"✅ Finnhub Success – Current ₹{data.get('c')}")
            self.last_source = "Finnhub"
            return df
        except Exception as e:
            print(f"❌ Finnhub failed: {e}")
            return None

    def fetch_yfinance(self, symbol, period="5d", interval="5m"):
        """Fetch from Yahoo Finance (Best for Indian stocks)"""
        print(f"\n🔹 Trying Yahoo Finance for {symbol}...")
        try:
            df = yf.download(symbol, period=period, interval=interval, progress=False)

            if df.empty:
                print("❌ Yahoo Finance returned no data")
                return None

            # Handle MultiIndex columns
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.get_level_values(0)

            # Normalize column names to lowercase
            df.columns = [col.lower() if isinstance(col, str) else str(col).lower() for col in df.columns]

            print(f"✅ Yahoo Finance Success – {len(df)} records")
            self.last_source = "Yahoo Finance"
            return df
        except Exception as e:
            print(f"❌ Yahoo Finance failed: {e}")
            return None

    def get_stock_data(self, symbol, prefer_source=None, **kwargs):
        """Fetch stock data with fallback logic"""
        fetchers = {
            'alpha_vantage': self.fetch_alpha_vantage,
            'polygon': self.fetch_polygon,
            'finnhub': self.fetch_finnhub,
            'yfinance': self.fetch_yfinance
        }

        # Try preferred source first
        if prefer_source and prefer_source in fetchers:
            df = fetchers[prefer_source](symbol, **kwargs)
            if df is not None and not df.empty:
                return self._normalize_dataframe(df)

        # Try all sources in order
        for name, fetcher in fetchers.items():
            if prefer_source == name:
                continue
            try:
                df = fetcher(symbol)
                if df is not None and not df.empty:
                    print(f"\n✅ Data fetched successfully from: {self.last_source}")
                    return self._normalize_dataframe(df)
            except Exception as e:
                print(f"❌ {name} failed: {e}")
                continue

        print("❌ All APIs failed – please check keys and connection.")
        return None

    def _normalize_dataframe(self, df):
        """Ensure consistent column names and format"""
        # Lowercase all columns
        df.columns = [col.lower() for col in df.columns]

        # Ensure required columns exist
        required_cols = ['open', 'high', 'low', 'close', 'volume']
        missing = [col for col in required_cols if col not in df.columns]
        if missing:
            print(f"⚠️ Warning: Missing columns: {missing}")

        return df


In [8]:
# ==========================================
# ADVANCED FEATURE ENGINEER CLASS
# ==========================================
class AdvancedFeatureEngineer:
    """Generate technical indicators for stock analysis"""

    def __init__(self):
        self.feature_names = []

    def add_all_features(self, df):
        """Add all technical indicators to dataframe"""
        df = df.copy()

        print("\n📊 Adding technical indicators...")

        try:
            df = self._add_sma_ema(df)
            print("✓ Moving averages added")
        except Exception as e:
            print(f"⚠️ Moving averages failed: {e}")

        try:
            df = self._add_macd(df)
            print("✓ MACD added")
        except Exception as e:
            print(f"⚠️ MACD failed: {e}")

        try:
            df = self._add_rsi(df)
            print("✓ RSI added")
        except Exception as e:
            print(f"⚠️ RSI failed: {e}")

        try:
            df = self._add_stochastic(df)
            print("✓ Stochastic added")
        except Exception as e:
            print(f"⚠️ Stochastic failed: {e}")

        try:
            df = self._add_adx(df)
            print("✓ ADX added")
        except Exception as e:
            print(f"⚠️ ADX failed: {e}")

        try:
            df = self._add_demarker(df)
            print("✓ DeMarker added")
        except Exception as e:
            print(f"⚠️ DeMarker failed: {e}")

        try:
            df = self._add_atr(df)
            print("✓ ATR added")
        except Exception as e:
            print(f"⚠️ ATR failed: {e}")

        try:
            df = self._add_bollinger(df)
            print("✓ Bollinger Bands added")
        except Exception as e:
            print(f"⚠️ Bollinger Bands failed: {e}")

        try:
            df = self._add_std_features(df)
            print("✓ Standard deviation added")
        except Exception as e:
            print(f"⚠️ Standard deviation failed: {e}")

        try:
            df = self._add_obv(df)
            print("✓ OBV added")
        except Exception as e:
            print(f"⚠️ OBV failed: {e}")

        try:
            df = self._add_accumulation_distribution(df)
            print("✓ A/D Line added")
        except Exception as e:
            print(f"⚠️ A/D Line failed: {e}")

        try:
            df = self._add_pivot_points(df)
            print("✓ Pivot points added")
        except Exception as e:
            print(f"⚠️ Pivot points failed: {e}")

        try:
            df = self._add_trendlines(df)
            print("✓ Trendlines added")
        except Exception as e:
            print(f"⚠️ Trendlines failed: {e}")

        try:
            df = self._add_market_profile(df)
            print("✓ Market profile added")
        except Exception as e:
            print(f"⚠️ Market profile failed: {e}")

        try:
            df = self._add_support_resistance(df)
            print("✓ Support/Resistance added")
        except Exception as e:
            print(f"⚠️ Support/Resistance failed: {e}")

        # Remove NaN rows - but keep enough data
        initial_rows = len(df)

        # First, try to drop rows where most columns are NaN
        threshold = len(df.columns) * 0.5
        df.dropna(thresh=threshold, inplace=True)

        # If too much data was removed, try a more lenient approach
        if len(df) < 50:
            print("⚠️ Too many NaNs detected, using forward fill...")
            df = df.copy()
            df.fillna(method='ffill', inplace=True)
            df.dropna(inplace=True)

        removed_rows = initial_rows - len(df)

        if len(df) == 0:
            print("\n❌ All data removed due to NaN values!")
            print("   This usually means insufficient data for indicators.")
            print("   Try using a longer time period (e.g., period='1y' or '2y')")
            return None

        print(f"\n✅ Feature engineering complete!")
        print(f"   Total features: {len(df.columns)}")
        print(f"   Rows removed (NaN): {removed_rows}")
        print(f"   Final dataset size: {len(df)} rows")

        return df

    def _add_sma_ema(self, df):
        """Simple and Exponential Moving Averages"""
        for period in [5, 10, 20, 50, 100, 200]:
            if len(df) >= period:
                df[f'sma_{period}'] = df['close'].rolling(window=period, min_periods=1).mean()
                df[f'ema_{period}'] = df['close'].ewm(span=period, adjust=False, min_periods=1).mean()
        return df

    def _add_macd(self, df):
        """MACD - Moving Average Convergence Divergence"""
        exp1 = df['close'].ewm(span=12, adjust=False).mean()
        exp2 = df['close'].ewm(span=26, adjust=False).mean()
        df['macd'] = exp1 - exp2
        df['macd_signal'] = df['macd'].ewm(span=9, adjust=False).mean()
        df['macd_hist'] = df['macd'] - df['macd_signal']
        return df

    def _add_rsi(self, df, period=14):
        """Relative Strength Index"""
        if len(df) < period:
            return df
        delta = df['close'].diff()
        gain = delta.where(delta > 0, 0).rolling(window=period, min_periods=1).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period, min_periods=1).mean()
        rs = gain / (loss + 1e-10)
        df[f'rsi_{period}'] = 100 - (100 / (1 + rs))
        return df

    def _add_stochastic(self, df, period=14):
        """Stochastic Oscillator"""
        if len(df) < period:
            return df
        low_min = df['low'].rolling(window=period, min_periods=1).min()
        high_max = df['high'].rolling(window=period, min_periods=1).max()
        df[f'stoch_k_{period}'] = 100 * (df['close'] - low_min) / (high_max - low_min + 1e-10)
        df[f'stoch_d_{period}'] = df[f'stoch_k_{period}'].rolling(window=3, min_periods=1).mean()
        return df

    def _add_adx(self, df, period=14):
        """Average Directional Index"""
        if len(df) < period:
            return df
        high, low, close = df['high'], df['low'], df['close']
        plus_dm = high.diff()
        minus_dm = -low.diff()
        plus_dm[plus_dm < 0] = 0
        minus_dm[minus_dm < 0] = 0

        tr = self._calculate_atr(df, period)
        plus_di = 100 * (plus_dm.rolling(window=period, min_periods=1).mean() / (tr + 1e-10))
        minus_di = 100 * (minus_dm.rolling(window=period, min_periods=1).mean() / (tr + 1e-10))
        dx = 100 * abs(plus_di - minus_di) / (plus_di + minus_di + 1e-10)
        df['adx'] = dx.rolling(window=period, min_periods=1).mean()
        return df

    def _add_demarker(self, df, period=14):
        """DeMarker Indicator"""
        if len(df) < period:
            return df
        demax = df['high'].diff()
        demin = df['low'].diff()
        demax = np.where(demax > 0, demax, 0)
        demin = np.where(demin < 0, abs(demin), 0)
        demax_sma = pd.Series(demax, index=df.index).rolling(window=period, min_periods=1).mean()
        demin_sma = pd.Series(demin, index=df.index).rolling(window=period, min_periods=1).mean()
        df['demarker'] = demax_sma / (demax_sma + demin_sma + 1e-10)
        return df

    def _add_atr(self, df, period=14):
        """Average True Range"""
        df['atr'] = self._calculate_atr(df, period)
        return df

    def _calculate_atr(self, df, period=14):
        """Helper function to calculate ATR"""
        tr1 = df['high'] - df['low']
        tr2 = abs(df['high'] - df['close'].shift())
        tr3 = abs(df['low'] - df['close'].shift())
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        return tr.rolling(window=period, min_periods=1).mean()

    def _add_bollinger(self, df, period=20):
        """Bollinger Bands"""
        if len(df) < period:
            return df
        sma = df['close'].rolling(window=period, min_periods=1).mean()
        std = df['close'].rolling(window=period, min_periods=1).std()
        df['bb_upper'] = sma + (2 * std)
        df['bb_lower'] = sma - (2 * std)
        df['bb_mid'] = sma
        df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (sma + 1e-10)
        return df

    def _add_std_features(self, df):
        """Standard Deviation Features"""
        for period in [10, 20, 50]:
            if len(df) >= period:
                df[f'std_{period}'] = df['close'].rolling(window=period, min_periods=1).std()
        return df

    def _add_obv(self, df):
        """On-Balance Volume"""
        obv = [0]
        for i in range(1, len(df)):
            if df['close'].iloc[i] > df['close'].iloc[i-1]:
                obv.append(obv[-1] + df['volume'].iloc[i])
            elif df['close'].iloc[i] < df['close'].iloc[i-1]:
                obv.append(obv[-1] - df['volume'].iloc[i])
            else:
                obv.append(obv[-1])
        df['obv'] = obv
        return df

    def _add_accumulation_distribution(self, df):
        """Accumulation/Distribution Line"""
        mfm = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'] + 1e-10)
        df['ad_line'] = (mfm * df['volume']).cumsum()
        return df

    def _add_pivot_points(self, df):
        """Pivot Points"""
        df['pivot'] = (df['high'] + df['low'] + df['close']) / 3
        df['r1'] = 2 * df['pivot'] - df['low']
        df['s1'] = 2 * df['pivot'] - df['high']
        df['r2'] = df['pivot'] + (df['high'] - df['low'])
        df['s2'] = df['pivot'] - (df['high'] - df['low'])
        return df

    def _add_trendlines(self, df, period=20):
        """Trendline slope via linear regression"""
        if len(df) < period:
            return df
        df['trend_slope'] = df['close'].rolling(window=period, min_periods=max(5, period//2)).apply(
            lambda x: np.polyfit(range(len(x)), x, 1)[0] if len(x) >= 2 else np.nan,
            raw=True
        )
        return df

    def _add_support_resistance(self, df, window=10):
        """Dynamic Support/Resistance Levels"""
        if len(df) < window:
            window = max(2, len(df) // 2)
        df['support'] = df['low'].rolling(window=window, min_periods=1).min()
        df['resistance'] = df['high'].rolling(window=window, min_periods=1).max()
        df['price_position'] = (df['close'] - df['support']) / (df['resistance'] - df['support'] + 1e-10)
        return df

    def _add_market_profile(self, df, bins=20):
        """Market Profile - Volume by Price"""
        try:
            df['price_bin'] = pd.qcut(df['close'], q=bins, duplicates='drop')
            volume_profile = df.groupby('price_bin')['volume'].sum()
            df['market_profile_strength'] = df['price_bin'].map(volume_profile)
        except Exception as e:
            df['price_bin'] = pd.cut(df['close'], bins=bins)
            volume_profile = df.groupby('price_bin')['volume'].sum()
            df['market_profile_strength'] = df['price_bin'].map(volume_profile)
        return df


In [9]:
# ==========================================
# VISUALIZATION FUNCTIONS
# ==========================================
def plot_price_ma(df, ticker):
    """Plot candlestick chart with moving averages"""
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='Price'
    ))

    ma_colors = {
        'sma_20': 'orange',
        'sma_50': 'blue',
        'sma_100': 'purple',
        'sma_200': 'red',
        'ema_20': 'yellow',
        'ema_50': 'cyan'
    }

    for ma, color in ma_colors.items():
        if ma in df.columns:
            ma_type = 'SMA' if 'sma' in ma else 'EMA'
            period = ma.split('_')[1]
            fig.add_trace(go.Scatter(
                x=df.index,
                y=df[ma],
                mode='lines',
                name=f'{ma_type} {period}',
                line=dict(color=color, width=2)
            ))

    fig.update_layout(
        title=f"{ticker} - Price with Moving Averages",
        xaxis_title='Date',
        yaxis_title='Price (₹)',
        template='plotly_dark',
        height=700,
        hovermode='x unified',
        xaxis_rangeslider_visible=False
    )
    fig.show()


def plot_comprehensive_dashboard(df, ticker):
    """Comprehensive dashboard with multiple indicators"""
    fig = make_subplots(
        rows=4, cols=2,
        shared_xaxes=True,
        subplot_titles=[
            f'{ticker} - Price & Moving Averages', 'Volume',
            'MACD', 'RSI',
            'Stochastic Oscillator', 'ADX (Trend Strength)',
            'Bollinger Bands', 'OBV'
        ],
        vertical_spacing=0.05,
        horizontal_spacing=0.1,
        specs=[
            [{"colspan": 2}, None],
            [{"type": "bar"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "scatter"}, {"type": "scatter"}]
        ],
        row_heights=[0.3, 0.23, 0.23, 0.24]
    )

    # Price with moving averages
    fig.add_trace(
        go.Candlestick(
            x=df.index, open=df['open'], high=df['high'],
            low=df['low'], close=df['close'], name='Price'
        ),
        row=1, col=1
    )

    for ma, color in [('sma_20', 'orange'), ('sma_50', 'blue'), ('ema_20', 'yellow')]:
        if ma in df.columns:
            fig.add_trace(
                go.Scatter(x=df.index, y=df[ma], mode='lines',
                          name=ma.upper(), line=dict(color=color, width=1)),
                row=1, col=1
            )

    # Volume
    colors = ['green' if df['close'].iloc[i] >= df['open'].iloc[i] else 'red'
              for i in range(len(df))]
    fig.add_trace(
        go.Bar(x=df.index, y=df['volume'], name='Volume',
               marker_color=colors, opacity=0.7),
        row=2, col=1
    )

    # RSI
    if 'rsi_14' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['rsi_14'], mode='lines',
                      name='RSI', line=dict(color='purple', width=2)),
            row=2, col=2
        )
        fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=2, opacity=0.5)
        fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=2, opacity=0.5)

    # MACD
    if 'macd' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['macd'], mode='lines',
                      name='MACD', line=dict(color='blue', width=2)),
            row=3, col=1
        )
    if 'macd_signal' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['macd_signal'], mode='lines',
                      name='Signal', line=dict(color='orange', width=2)),
            row=3, col=1
        )

    # Stochastic
    if 'stoch_k_14' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['stoch_k_14'], mode='lines',
                      name='%K', line=dict(color='blue', width=2)),
            row=3, col=2
        )
    if 'stoch_d_14' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['stoch_d_14'], mode='lines',
                      name='%D', line=dict(color='orange', width=2)),
            row=3, col=2
        )
        fig.add_hline(y=80, line_dash="dash", line_color="red", row=3, col=2, opacity=0.5)
        fig.add_hline(y=20, line_dash="dash", line_color="green", row=3, col=2, opacity=0.5)

    # ADX
    if 'adx' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['adx'], mode='lines',
                      name='ADX', line=dict(color='cyan', width=2)),
            row=4, col=1
        )
        fig.add_hline(y=25, line_dash="dash", line_color="orange", row=4, col=1)

    # OBV
    if 'obv' in df.columns:
        fig.add_trace(
            go.Scatter(x=df.index, y=df['obv'], mode='lines',
                      name='OBV', line=dict(color='orange', width=2)),
            row=4, col=2
        )

    fig.update_layout(
        title=f"{ticker} - Comprehensive Technical Analysis Dashboard",
        template='plotly_dark',
        height=1400,
        showlegend=False,
        hovermode='x unified'
    )

    fig.update_xaxes(rangeslider_visible=False)
    fig.show()


In [10]:
# ==========================================
# ANALYSIS FUNCTIONS
# ==========================================
def generate_trading_signals(df):
    """Generate trading signals based on technical indicators"""
    signals = {
        'timestamp': df.index[-1],
        'close_price': df['close'].iloc[-1],
        'signals': [],
        'overall_sentiment': 'NEUTRAL'
    }

    bullish_count = 0
    bearish_count = 0

    # RSI Signal
    if 'rsi_14' in df.columns:
        rsi = df['rsi_14'].iloc[-1]
        if rsi < 30:
            signals['signals'].append({'indicator': 'RSI', 'signal': 'BUY', 'value': rsi, 'reason': 'Oversold'})
            bullish_count += 2
        elif rsi > 70:
            signals['signals'].append({'indicator': 'RSI', 'signal': 'SELL', 'value': rsi, 'reason': 'Overbought'})
            bearish_count += 2
        else:
            signals['signals'].append({'indicator': 'RSI', 'signal': 'NEUTRAL', 'value': rsi, 'reason': 'Normal range'})

    # MACD Signal
    if 'macd' in df.columns and 'macd_signal' in df.columns:
        macd = df['macd'].iloc[-1]
        macd_signal = df['macd_signal'].iloc[-1]

        if macd > macd_signal and df['macd'].iloc[-2] <= df['macd_signal'].iloc[-2]:
            signals['signals'].append({'indicator': 'MACD', 'signal': 'BUY', 'value': macd - macd_signal, 'reason': 'Bullish crossover'})
            bullish_count += 2
        elif macd < macd_signal and df['macd'].iloc[-2] >= df['macd_signal'].iloc[-2]:
            signals['signals'].append({'indicator': 'MACD', 'signal': 'SELL', 'value': macd - macd_signal, 'reason': 'Bearish crossover'})
            bearish_count += 2
        elif macd > macd_signal:
            signals['signals'].append({'indicator': 'MACD', 'signal': 'BULLISH', 'value': macd - macd_signal, 'reason': 'Above signal line'})
            bullish_count += 1
        else:
            signals['signals'].append({'indicator': 'MACD', 'signal': 'BEARISH', 'value': macd - macd_signal, 'reason': 'Below signal line'})
            bearish_count += 1

    # Moving Average Signal
    if 'sma_20' in df.columns and 'sma_50' in df.columns:
        sma_20 = df['sma_20'].iloc[-1]
        sma_50 = df['sma_50'].iloc[-1]
        close = df['close'].iloc[-1]

        if sma_20 > sma_50 and close > sma_20:
            signals['signals'].append({'indicator': 'MA', 'signal': 'BUY', 'value': None, 'reason': 'Golden cross + price above'})
            bullish_count += 1
        elif sma_20 < sma_50 and close < sma_20:
            signals['signals'].append({'indicator': 'MA', 'signal': 'SELL', 'value': None, 'reason': 'Death cross + price below'})
            bearish_count += 1

    # Bollinger Bands Signal
    if 'bb_upper' in df.columns and 'bb_lower' in df.columns:
        close = df['close'].iloc[-1]
        bb_upper = df['bb_upper'].iloc[-1]
        bb_lower = df['bb_lower'].iloc[-1]

        if close <= bb_lower:
            signals['signals'].append({'indicator': 'Bollinger', 'signal': 'BUY', 'value': None, 'reason': 'Price at lower band'})
            bullish_count += 1
        elif close >= bb_upper:
            signals['signals'].append({'indicator': 'Bollinger', 'signal': 'SELL', 'value': None, 'reason': 'Price at upper band'})
            bearish_count += 1

    # Overall Sentiment
    if bullish_count > bearish_count + 1:
        signals['overall_sentiment'] = 'BULLISH'
    elif bearish_count > bullish_count + 1:
        signals['overall_sentiment'] = 'BEARISH'
    else:
        signals['overall_sentiment'] = 'NEUTRAL'

    signals['bullish_score'] = bullish_count
    signals['bearish_score'] = bearish_count

    return signals


def print_trading_signals(signals):
    """Print trading signals in a formatted way"""
    print(f"\n{'='*60}")
    print("🎯 TRADING SIGNALS")
    print(f"{'='*60}")
    print(f"Timestamp: {signals['timestamp']}")
    print(f"Current Price: ₹{signals['close_price']:.2f}")
    print(f"\nOverall Sentiment: {signals['overall_sentiment']}")
    print(f"Bullish Score: {signals['bullish_score']} | Bearish Score: {signals['bearish_score']}")
    print(f"\n{'Indicator':<15} {'Signal':<10} {'Value':<15} {'Reason'}")
    print("-" * 60)

    for sig in signals['signals']:
        value_str = f"{sig['value']:.2f}" if sig['value'] is not None else "N/A"
        signal_emoji = {
            'BUY': '🟢', 'SELL': '🔴', 'BULLISH': '📈',
            'BEARISH': '📉', 'NEUTRAL': '⚪', 'INFO': 'ℹ️'
        }.get(sig['signal'], '')

        print(f"{sig['indicator']:<15} {signal_emoji} {sig['signal']:<8} {value_str:<15} {sig['reason']}")


def analyze_stock(ticker, period='1y', interval='1d', add_indicators=True, plot_charts=True):
    """Main function to analyze a stock"""
    # Validate ticker
    ticker = ticker.upper()
    stock_info = get_stock_info(ticker)

    print(f"\n{'='*60}")
    print(f"📈 STOCK ANALYSIS: {ticker}")
    if stock_info:
        print(f"Company: {stock_info['name']}")
        print(f"Sector: {stock_info['sector']}")
    print(f"{'='*60}")

    # Fetch data
    fetcher = StockDataFetcher(api_keys=API_KEYS)
    df = fetcher.get_stock_data(ticker, prefer_source='yfinance', period=period, interval=interval)

    if df is None or df.empty:
        print("\n❌ Failed to fetch stock data. Please check ticker symbol and try again.")
        return None

    print(f"\n✅ Fetched {len(df)} rows of data from {df.index[0]} to {df.index[-1]}")

    # Add technical indicators
    if add_indicators:
        engineer = AdvancedFeatureEngineer()
        df = engineer.add_all_features(df)

    if df is None or len(df) == 0:
        print("\n❌ No data available after processing.")
        return None

    # Display summary
    print(f"\n{'='*60}")
    print("📊 SUMMARY STATISTICS")
    print(f"{'='*60}")
    print(f"Latest Close Price: ₹{df['close'].iloc[-1]:.2f}")
    print(f"Period High: ₹{df['high'].max():.2f}")
    print(f"Period Low: ₹{df['low'].min():.2f}")
    print(f"Average Volume: {df['volume'].mean():,.0f}")

    if 'rsi_14' in df.columns:
        latest_rsi = df['rsi_14'].iloc[-1]
        print(f"Latest RSI (14): {latest_rsi:.2f}")
        if latest_rsi > 70:
            print("  ⚠️ Overbought condition")
        elif latest_rsi < 30:
            print("  ⚠️ Oversold condition")

    # Generate visualizations
    if plot_charts:
        print(f"\n📈 Generating dashboard...")
        plot_comprehensive_dashboard(df, ticker)

    print(f"\n{'='*60}")
    print("✅ ANALYSIS COMPLETE")
    print(f"{'='*60}\n")

    return df


def analyze_sector(sector, period='6mo', interval='1d'):
    """Analyze all stocks in a sector"""
    sector = sector.upper()

    if sector not in INDIAN_STOCKS:
        print(f"❌ Sector '{sector}' not found.")
        list_sectors()
        return None

    stocks = INDIAN_STOCKS[sector]
    results = {}

    print(f"\n{'='*60}")
    print(f"🏭 ANALYZING {sector} SECTOR ({len(stocks)} stocks)")
    print(f"{'='*60}\n")

    for ticker, name in stocks.items():
        print(f"\n{'='*40}")
        print(f"Analyzing: {name} ({ticker})")
        print(f"{'='*40}")

        df = analyze_stock(ticker, period=period, interval=interval,
                          add_indicators=True, plot_charts=False)

        if df is not None:
            signals = generate_trading_signals(df)
            results[ticker] = {
                'name': name,
                'signals': signals,
                'df': df
            }

    # Summary
    print(f"\n{'='*60}")
    print(f"📊 {sector} SECTOR SUMMARY")
    print(f"{'='*60}\n")
    print(f"{'Ticker':<15} {'Company':<30} {'Price':<12} {'Sentiment':<12} {'Score'}")
    print("-" * 80)

    for ticker, data in results.items():
        sig = data['signals']
        score_str = f"{sig['bullish_score']}/{sig['bearish_score']}"
        print(f"{ticker:<15} {data['name']:<30} ₹{sig['close_price']:<10.2f} {sig['overall_sentiment']:<12} {score_str}")

    return results


In [ ]:
# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":

    print("\n" + "="*60)
    print("🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL")
    print("="*60)

    # Show available functions
    print("\n📚 Available Functions:")
    print("  list_sectors()                    - Show all sectors")
    print("  list_stocks_by_sector('IT')       - List stocks in a sector")
    print("  search_stock('RELIANCE')          - Search for a stock")
    print("  analyze_stock('RELIANCE.NS')      - Analyze a stock")
    print("  analyze_sector('IT')              - Analyze entire sector")

    # : Analyze a single stock
    print("\n" + "="*60)
    print(" Single Stock Analysis")
    print("="*60)

    df = analyze_stock('RELIANCE.NS', period='6mo', interval='1d',
                      add_indicators=True, plot_charts=True)

    if df is not None:
        signals = generate_trading_signals(df)
        print_trading_signals(signals)

    #  Search and analyze
    print("\n" + "="*60)
    print(" Search and Analyze")
    print("="*60)

    search_stock('TCS')

    #  List sectors
    print("\n" + "="*60)
    print(" Available Sectors")
    print("="*60)
    list_sectors()

    # Interactive mode
    print("\n" + "="*60)
    print("🎮 INTERACTIVE MODE")
    print("="*60)
    print("\nCommands:")
    print("  analyze <TICKER>  - Analyze a stock (e.g., analyze SBIN.NS)")
    print("  search <QUERY>    - Search stocks (e.g., search HDFC)")
    print("  sector <SECTOR>   - Analyze sector (e.g., sector IT)")
    print("  list              - List all sectors")
    print("  quit              - Exit")

    while True:
        try:
            command = input("\n> ").strip().lower()

            if command == 'quit':
                break
            elif command == 'list':
                list_sectors()
            elif command.startswith('search '):
                query = command.split(' ', 1)[1]
                search_stock(query)
            elif command.startswith('analyze '):
                ticker = command.split(' ', 1)[1].upper()
                df = analyze_stock(ticker, period='6mo', interval='1d',
                                  add_indicators=True, plot_charts=True)
                if df is not None:
                    signals = generate_trading_signals(df)
                    print_trading_signals(signals)
            elif command.startswith('sector '):
                sector = command.split(' ', 1)[1].upper()
                analyze_sector(sector, period='6mo', interval='1d')
            else:
                print("❌ Unknown command. Type 'quit' to exit.")
        except KeyboardInterrupt:
            print("\n\nExiting...")
            break
        except Exception as e:
            print(f"❌ Error: {e}")


🇮🇳 INDIAN STOCK MARKET ANALYSIS TOOL

📚 Available Functions:
  list_sectors()                    - Show all sectors
  list_stocks_by_sector('IT')       - List stocks in a sector
  search_stock('RELIANCE')          - Search for a stock
  analyze_stock('RELIANCE.NS')      - Analyze a stock
  analyze_sector('IT')              - Analyze entire sector

 Single Stock Analysis

📈 STOCK ANALYSIS: RELIANCE.NS
Company: Reliance Industries
Sector: NIFTY_50

🔹 Trying Yahoo Finance for RELIANCE.NS...
✅ Yahoo Finance Success – 127 records

✅ Fetched 127 rows of data from 2025-04-09 00:00:00 to 2025-10-09 00:00:00

📊 Adding technical indicators...
✓ Moving averages added
✓ MACD added
✓ RSI added
✓ Stochastic added
✓ ADX added
✓ DeMarker added
✓ ATR added
✓ Bollinger Bands added
✓ Standard deviation added
✓ OBV added
✓ A/D Line added
✓ Pivot points added
✓ Trendlines added
✓ Market profile added
✓ Support/Resistance added

✅ Feature engineering complete!
   Total features: 44
   Rows removed (NaN): 0


✅ ANALYSIS COMPLETE


🎯 TRADING SIGNALS
Timestamp: 2025-10-09 00:00:00
Current Price: ₹1377.30

Overall Sentiment: NEUTRAL
Bullish Score: 0 | Bearish Score: 1

Indicator       Signal     Value           Reason
------------------------------------------------------------
RSI             ⚪ NEUTRAL  36.62           Normal range
MACD            📉 BEARISH  -0.09           Below signal line

 Search and Analyze

🔍 SEARCH RESULTS FOR: 'TCS'
TCS.NS               - Tata Consultancy Services

 Available Sectors

📊 AVAILABLE INDIAN STOCK SECTORS

NIFTY_50: 50 stocks

BANKING: 12 stocks

IT: 9 stocks

PHARMA: 9 stocks

AUTO: 9 stocks

FMCG: 9 stocks

ENERGY: 10 stocks

METALS: 8 stocks

INFRA: 7 stocks

TELECOM: 2 stocks

MIDCAP: 9 stocks

Total Indian Stocks: 92

🎮 INTERACTIVE MODE

Commands:
  analyze <TICKER>  - Analyze a stock (e.g., analyze SBIN.NS)
  search <QUERY>    - Search stocks (e.g., search HDFC)
  sector <SECTOR>   - Analyze sector (e.g., sector IT)
  list              - List all se